In this notebook, we demonstrate the techniques and procedures used to study matroids of rank 4 on 8 elements. These techniques scale in a straightforward way to our analysis of (4,9)-matroids.

In [37]:
using Oscar
using Combinatorics
pm = Polymake


include("../src/matroid_realization.jl");
include("../src/old/reduceIdealsv2.jl")
include("../src/old/JacobianCriterion.jl")
include("../src/old/TSC_CoordRingV2.jl")

d4n8 = vec(readlines("../data/d4n8.dat"))

LoadError: SystemError: opening file "../data/d4n8.dat": No such file or directory

We begin by removing matroids whose realization spaces are known to be smooth. We apply the following lemma:

**Lemma:** Let $M$ be a $(4,8)$-matroid such that there exists an element $i\in [8]$ contained in exactly $0\leq k\leq 3$ rank $3$ flats of cardinality at least $4$. Then there exists a smooth morphism $Gr_{M}\to Gr_{M|[8]\setminus i}$.

**Remark:** Matroid duality induces an isomorphism $Gr_{M}\cong Gr_{M^*}$. In other words, $Gr_{M|[8]\setminus i}\cong Gr_{(M|[8]\setminus i)^*}$. By prior work, we know that all $(3,7)$ matroids have smooth thin Schubert cells, and therefore smooth realization spaces. Hence, we can restrict our search for singular realization spaces to those $(4,8)$ matroids that do not satisfy the hypotheses of the above lemma.

We begin by isolating $(4,8)$-matroids such that every element of $[8]$ is contained in at least $4$ rank $3$ flats of cardinality at least $4$.

In [6]:
function in_planes(n,L)

    Ln = [l for l in L if n in l]

    return(length(Ln))

end

println("isolate matroids with such that every element of [1,8] is in at least 4 planes")

for t in 1:length(d4n8)
    
   # println(t)

    Mt = matroid_from_revlex_basis_encoding(d4n8[t],4,8)

    L = [h for h in hyperplanes(Mt) if length(h)>3]

    ns = [n for n in 1:8 if in_planes(n,L)>3]

    if length(ns) == 8
                #println("four planes")
                    open("4_8_four_planes.dat", "a") do file
                         write(file,d4n8[t],"\n")
                    end 
    end
end

isolate matroids with such that every element of [1,8] is in at least 4 planes


We have thus significantly cut down the number of matroids we need to consider:

In [7]:
four_planes = vec(readlines("4_8_four_planes.dat"))

95-element Vector{String}:
 "0********0**********0********0***********0********0*************00****"
 "0********0**********0********0***********0******0***************0**0**"
 "0********0**********0*******0************0********0*************0*0***"
 "0********0**********0*******0************0*******0**************0**0**"
 "0********0**********0*******0************0*******0**********0********0"
 "0********0****0*****0****************************0*******0******00****"
 "0********0****0*****0****************************0*****0********0**0**"
 "0********0****0*****0****************************0*****0****0********0"
 "0********0****0*****0********0***********0******0***************0**0**"
 "0********0****0*****0********0***********0******0***********0********0"
 "0********0****0*****0********0***********0******0******0********0*****"
 "0********0****0*****0********0***********0******0******0********0**0**"
 "0********0****0*****0**0**********************0**********00**********0"
 ⋮
 "000000

We now check the remaining matroids to see if their realization spaces are singular. Note that this computation relies on the existence of a circuit of cardinality $n+1$, called a *reference circuit*. Therefore, the first step of the following computation checks for the existence of such a circuit. Matroids without such a reference circuit will be studied via their thin Schubert cells. The data for the coordinate rings consists of a polynomial ring, a multiplicative semigroup, and the vanishing ideal.

We sort the matroids into the following categories: 

1) Not realizable.

2) Vanishing idel reduces to 0.

3) Vanishing ideal is univariate and principle.

4) Vanishing ideal is multivariate and principle.

5) Vanishing ideal is multivariate and principle.

6) Vanishing ideal is multivariate with multiple generators.

7) No reference Circuit

**Note:** Different reference circuits lead to difference presenation of the coordinate ring of the realization space. Of course all presentations yield isomorphic realization spaces. However, we keep track of the reference circuit used for computing the coordinate ring using the following function.

In [8]:
function set_to_stars(set,n)
    
    z = []
    
    for a in 1:n
        if a in set
            push!(z,"*")
        else
            push!(z,"0")
        end
    end
    return join(z)
end


set_to_stars (generic function with 1 method)

Then we save the data as a string of $*$'s and $0$'s, such that the first $8$ entries of the string tell us the reference circuit. That is, we have a "$*$" in the $i$th position if $i$ is an element of the referenc circuit, "$0$" otherwise. The remaining $70$ entries of the string correspond to the the elements of ${[8]\choose 4}$ listed in reverse lex order.

Hence, we classify each matroid into one of the 7 categories above, and record which reference basis give us such a presentation of the coordinate ring. Then, where applicable, we check if the realization space is singular or smooth.

In [9]:
for z in 1:length(four_planes)
    ##printn(z)
    Mz = matroid_from_revlex_basis_encoding(four_planes[z], 4, 8)
    #printn("check reference circuits")

    C = [c for c in circuits(Mz) if length(c) == rank(Mz) + 1]

if length(C) == 0
   # #printn("no_ref_circuit")

     open("4_planes_no_ref_circuit_4_8.dat", "a") do file
            write(file,four_planes[z],"\n")
        end

else

    I = matroid_to_reduced_expression_min_chart(Mz, QQ)
    data = join([set_to_stars(I[3],8),four_planes[z]])

 #check realizability   
    if I isa String    
        #printn("not realizable")
        open("not_realizable_4_8.dat", "a") do file
            write(file,data,"\n")
        end

    elseif isone(ideal(I[1])) 
               #printn("not realizable unit ideal")

               open("not_realizable_4_8.dat", "a") do file
                     write(file,data,"\n")
                end

    elseif (1 in I[1] || -1 in I[1])
            #printn("not realizable 1 or -1")
            open("not_realizable_4_8.dat", "a") do file
                write(file,data,"\n")
            end  
#classify if realizable
    elseif (length(I[1]) == 0||I[1] == [0]) #reduces to 0 
            #printn("zero ideal")
            open("reduce_to_zero_4_8.dat", "a") do file
            write(file,data,"\n")
        end   
 
   # elseif int_gens(I[1])#has integral generator. This only matters if reducing over ZZ

                #printn("Computing Jacobian ideal with saturation")
    #            J = simplified_2_singular_locus(I[1],I[2],true)   

     #       if !(isone(J))
                 #printn("int gen singular")
      #              open("int_generator_4_8/singular.dat", "a") do file
       #                  write(file,data,"\n")
        #         end 
         #   else
                #printn("int gen smooth")
          #          open("int_generator_4_8/smooth.dat", "a") do file
           #              write(file,data,"\n")
            #        end
             #end
  

    elseif length(I[1]) == 1
        if (length(vars(I[1][1])) == 1)
                  #printn("Computing Jacobian ideal with saturation")
                    J = simplified_2_singular_locus(I[1],I[2],true)   
              if !(isone(J))
                     #printn("princ univariate singular")
                        open("princ_univariate_4_8_singular.dat", "a") do file
                             write(file,data,"\n")
                         end 
              else
                    #printn("princ univariate smooth")
                      open("princ_univariate_4_8_smooth.dat", "a") do file
                          write(file,data,"\n")
                        end
              end

            
        else
                 #printn("Computing Jacobian ideal with saturation")
                 J = simplified_2_singular_locus(I[1],I[2],true)   
             if !(isone(J))
                 #printn("princ multivariate singular")
                    open("princ_multivariate_4_8_singular.dat", "a") do file
                         write(file,data,"\n")
                 end 
            else
                #printn("princ multivariate smooth")
                    open("princ_multivariate_4_8_smooth.dat", "a") do file
                         write(file,data,"\n")
                    end
            end 
        end
    elseif length(I[1])>1
        
        Ivars = ideal_vars(I[1]) 
        if length(Ivars) == 1
                 ##printn("not principal univariate")#univariate ideal with multiple generators
                        #printn("Computing Jacobian ideal with saturation")
                      J = simplified_2_singular_locus(I[1],I[2],true)   
                   if !(isone(J))
                     #printn("not principle univariate singular")
                        open("np_univariate_4_8_singular.dat", "a") do file
                             write(file,data,"\n")
                          end 
                    else
                     #printn("not principle univariate smooth")
                      open("np_univariate_4_8_smooth.dat", "a") do file
                          write(file,data,"\n")
                        end
                    end
        else    
            ##printn("not principle multivariate")#multivariate ideal with multiple generators
                  #printn("Computing Jacobian ideal with saturation")
                  J = simplified_2_singular_locus(I[1],I[2],true)   
             if !(isone(J))
                 #printn("np multivariate singular")
                    open("nprinc_multivariate_4_8_singular.dat", "a") do file
                         write(file,data,"\n")
                    end 
            else
                #printn("not principal multivariate smooth")
                    open("nprinc_multivariate_4_8_smooth.dat", "a") do file
                         write(file,data,"\n")
                    end
             end
        end
        
    else
       #printn("to net")#to make sure sorting is exhaustive
       open("net_4_8.dat", "a") do file
            write(file,data,"\n")
     end
end
end
end


We study the four remaining matroids, those which do not have a reference circuit. We do so at the level of thin Schubert cells.

In [5]:
no_circs = vec(readlines("4_planes_no_ref_circuit_4_8.dat"))

4-element Vector{String}:
 "0********0****0*****0**0***00************00***0**0*****0****0********0"
 "000000000******0000************00000000************0000******000000000"
 "0000000000000000000000000**********0000000000********************00000"
 "000000000000000000000000000000000000******0******0***0******0*0**0****"

In [33]:
for i in 1:4
    Mi = matroid_from_revlex_basis_encoding(no_circs[i],4,8)
    T = matroid_to_reduced_TSC_min_basis(Mi,QQ)
    
    print("matroid ",i," : ", T,"\n\n")
end

matroid 1 : (fmpq_mpoly[1], fmpq_mpoly[x_{3, 1}, x_{2, 4}, x_{1, 3}, x_{4, 4}, x_{4, 2}, x_{4, 3}, x_{3, 4}], [1, 2, 3, 5])

matroid 2 : (fmpq_mpoly[0], fmpq_mpoly[x_{2, 1}, x_{1, 1}, x_{2, 2}, x_{1, 2}, x_{1, 1}*x_{2, 2} - x_{2, 1}*x_{1, 2}, x_{4, 3}, x_{3, 3}, x_{4, 4}, x_{3, 4}, x_{3, 3}*x_{4, 4} - x_{4, 3}*x_{3, 4}], [1, 2, 5, 6])

matroid 3 : (fmpq_mpoly[0], fmpq_mpoly[x_{2, 1}, x_{1, 1}, x_{2, 2}, x_{1, 2}, x_{1, 1}*x_{2, 2} - x_{2, 1}*x_{1, 2}, x_{2, 3}, x_{1, 3}, x_{1, 1}*x_{2, 3} - x_{2, 1}*x_{1, 3}, x_{1, 2}*x_{2, 3} - x_{2, 2}*x_{1, 3}, x_{4, 4}, x_{3, 4}], [1, 2, 6, 7])

matroid 4 : (fmpq_mpoly[1], fmpq_mpoly[x_{2, 1}, x_{1, 4}, x_{3, 2}, x_{3, 3}, x_{2, 4}, x_{3, 4}], [1, 2, 4, 8])



Observe that the vanishing ideal of the thin Schubert cells for matroids 1 and 4 are the unit ideal. Hence these matroids are not realizable. For matroids 2 and 3, the vanishing ideal is trivial. Hence, all realization spaces for $(4,8)$ matoids are nonsingular.